In [5]:
            ########### Asks user to input file name, start/end date, timeframe and filters the csv out as per request. 
            ########### Saves to new csv
import pandas as pd
import numpy as np
from datetime import datetime

#function to validate the Start Date
def GetStartDate():
    isValid=False
    while not isValid:
        userIn = input('Enter Start date(yyyy/mm/dd): ')
        try: # strptime throws an exception if the input doesn't match the pattern
            sd = datetime.strptime(userIn, "%Y/%m/%d")
            isValid=True
        except:
            print("Invalid date. Try again in format: yyyy/mm/dd")
    return sd


# function to insert the End Date or leave current date
def GetEndDate():

    userIn = str(input('Enter End date(yyyy/mm/dd) OR Press "x" to proceed with now-date: '))
    if userIn ==str('x'): # strptime throws an exception if the input doesn't match the pattern
        ed = datetime.now()
    else:
        ed = datetime.strptime(userIn, "%Y/%m/%d")     
    return ed

#Input variables
file = str(input('Enter File Name ex: LTC.csv '))
start_date = GetStartDate() #validates date input
end_date = GetEndDate()  #returns either custom date or now
time_frame = str(input('Enter Timeframe to filter in Minutes. Ex: 1/5/15/60 '))


df = pd.read_csv(file, dtype=str, skiprows=1)    #skips first row in csv which has hyperlink
df['unix'] = pd.to_datetime(df['unix'], unit='ms')
df['date'] = pd.to_datetime(df['date']) 

#filtering between 2 dates
after_start_date = df["date"] >= start_date
before_end_date = df["date"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates = df.loc[between_two_dates]

#set timeframe as per input n mins
df = (df.set_index('unix')
        .resample(time_frame + 'T').first() #5T = 5min, 1T - 1min
        .reset_index()
       .reindex(columns=df.columns))
df['unix'] = df['unix'].astype(np.int64) // 10**9

sym = df['symbol'][3]
symbol = sym.replace('/', '_')
str_start = start_date.strftime("%Y%m%d")
str_end = end_date.strftime("%Y%m%d")

df.to_csv(symbol + time_frame + '_'+ str_start + str_end +'.csv')
# print(my_date)
# print(end_date)

Enter File Name ex: LTC.csv LCT.csv
Enter Start date(yyyy/mm/dd): 2021/10/30
Enter End date(yyyy/mm/dd) OR Press "x" to proceed with now-date: x
Enter Timeframe to filter in Minutes. Ex: 1/5/15/60 60


In [1]:
              ################ Backtest: Rearrange raw data from 1 min to 5 min

import pandas as pd
import numpy as np
import os
file = 'ETH.csv'
df = pd.read_csv(file, dtype=str)

df['unix'] = pd.to_datetime(df['unix'], unit='ms')

df = (df.set_index('unix')
        .resample('5T').first() #5T = 5min, 1T - 1min
        .reset_index()
       .reindex(columns=df.columns))
df['unix'] = df['unix'].astype(np.int64) // 10**9
print (df.head())
df.to_csv('ETH_5min.csv')

         unix                 date    symbol open high  low close Volume ETH  \
0  1574840700  2019-11-27 07:45:00  ETH/USDT  146  146  146   146      0.010   
1  1574841000  2019-11-27 07:50:00  ETH/USDT  146  146  146   146          0   
2  1574841300  2019-11-27 07:55:00  ETH/USDT  146  146  146   146          0   
3  1574841600  2019-11-27 08:00:00  ETH/USDT  146  146  146   146          0   
4  1574841900  2019-11-27 08:05:00  ETH/USDT  146  146  146   146          0   

  Volume USDT tradecount  
0     1.46000          1  
1           0          0  
2           0          0  
3           0          0  
4           0          0  
